# Redes Neurais

## Criando uma rede neural

A typical training procedure for a neural network is as follows:

1. **Define the neural network that has some learnable parameters (or weights)**
2. **Iterate over a dataset of inputs**
3. **Process input through the network**
4. **Compute the loss (how far is the output from being correct)**
5. Propagate gradients back into the network’s parameters
6. Update the weights of the network, typically using a simple update rule: weight = weight - lr * gradient

Nós definimos manualmente um classificador linear, criando dois pesos (w_a e w_b) para serem aprendidos e atualizando eles com gradiente descendente manualmente. Agora vamos ver como o PyTorch nos oferece uma abstração em módulos, para nos preocupamos paenas com a definição da nossa rede neural (a nossa reta de antes)


In [1]:
import torch
import torch.nn as nn


# Se uma classe herda de nn.Module ela é considerada um módulo do pytorch

class Net(nn.Module):  # uma MLP com uma hidden layer!

    def __init__(self, num_features, hidden_size, nb_classes):
        super(Net, self).__init__()
        self.in_layer = nn.Linear(num_features, hidden_size)
        self.out_layer = nn.Linear(hidden_size, nb_classes)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.in_layer(x)
        x = torch.relu(x)
        x = self.out_layer(x)
        x = torch.sigmoid(x)
        return x
    
#     def backward(self, x):
#         #  não precisa implementar isso
#         pass

net = Net(64*64, 120, 10)
print(net)

Net(
  (in_layer): Linear(in_features=4096, out_features=120, bias=True)
  (out_layer): Linear(in_features=120, out_features=10, bias=True)
)


Os parâmetros (pesos) que devem ser treinados para nossa rede (módulo) pode ser acessado com `.parameters()`

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # in_layer size

4
torch.Size([120, 4096])


Bora jogar uma entrada aleatória na rede, mesmo sem treinar, só pra ver o ue acontece

In [3]:
input = torch.randn(1, 64*64)  # primeira dim é o batch_size
out = net(input)
print(out)
print(torch.max(out, dim=-1))

tensor([[0.5442, 0.5215, 0.5510, 0.4930, 0.4472, 0.6177, 0.5531, 0.4250, 0.5107,
         0.5734]], grad_fn=<SigmoidBackward>)
(tensor([0.6177], grad_fn=<MaxBackward0>), tensor([5]))


## Função de custo

Definir uma função de custo em pytorch é muito simples! As mais usadas na literatura já estão implementadas e basta chamar seu nome. Caso deseja criar uma nova, basta definir o `forward` e o resto será feito pelo pytorch! Vamos dar uma olhada nisso com exemplos

In [4]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
my_loss = nn.MSELoss()

lista completa de todas as losses: 
https://pytorch.org/docs/stable/nn.html#loss-functions

In [5]:
loss = my_loss(output, target)
loss

tensor(0.8440, grad_fn=<MseLossBackward>)

Se seguir o backprop pra loss, veremos que o grafo será o seguinte:

input -> linear -> relu -> linear -> sigmoid
      -> MSELoss
      -> loss

Backpropagation agora é como anteriormente! Só chamar loss.backward()

In [6]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('in_layer.bias.grad before backward')
print(net.in_layer.bias.grad)

loss.backward()

print('in_layer.bias.grad after backward')
print(net.in_layer.bias.grad)

in_layer.bias.grad before backward
None
in_layer.bias.grad after backward
tensor([ 1.2219e-02, -1.9659e-05,  0.0000e+00,  6.9967e-03,  6.0378e-03,
         4.3230e-03,  2.2502e-03,  0.0000e+00,  2.2966e-03, -5.4992e-03,
         0.0000e+00,  0.0000e+00, -2.8742e-03,  6.0687e-03,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -3.0687e-03,  9.0070e-03,
        -1.3563e-03,  2.4511e-03,  6.2430e-03,  0.0000e+00,  0.0000e+00,
        -8.8887e-03,  0.0000e+00,  0.0000e+00,  1.3174e-02, -6.2736e-03,
         0.0000e+00,  8.3735e-03,  0.0000e+00,  1.8505e-03,  1.4167e-02,
         0.0000e+00, -1.1964e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -1.7655e-03,  6.8359e-03,  8.7744e-03,
        -4.2429e-03,  0.0000e+00,  0.0000e+00, -9.3655e-06, -9.4452e-03,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -2.5224e-03,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -1.2378e-02,  0.0000e+00,  5.8981e-03,
        -9.1476e-05,  1.0832e-02,  0.0000e+00,  0.

## Atualizando os pesos (Optimizer)

Na nossa regressão linear estavamos atualizando os pesos com a famosa regra "delta"

    wi = wi - lr*wprev 

Mas existe uma gama de algoritmos que procuram atualizar os pesos de uma forma mais inteligente:

<img src="http://cs231n.github.io/assets/nn3/opt2.gif" width="45%" style="float:left;" />
<img src="http://cs231n.github.io/assets/nn3/opt1.gif" width="45%" style="float:left;" />

In [7]:
# regra delta manualmente:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = my_loss(output, target)
loss.backward()
optimizer.step()    # Does the update

## Treinamento

Basta seguir o passo acima por um número de épocas!

In [13]:
my_loss = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)  # olha a documentação os outros parâmetros que cada optimizer tem!
dataset = []
  
for _ in range(10):
    input = torch.randn(1, 64*64) # random input
    target = torch.randn(10).view(1, -1) # a dummy target, for example and make it the same shape as output
    dataset.append((input, target))

for epoch in range(100):
    for input, target in dataset:
        output = net(input)
        loss = my_loss(output, target)
        loss.backward()
        optimizer.step()    
        
with torch.no_grad():
    c = 0
    for input, target in dataset:
        output = net(input)
        if torch.argmax(output) == torch.argmax(target):
            c += 1
    print('Acc: ', c/len(dataset))

Acc:  0.1
